In [1]:
# A basic demonstration of the character-level tokenizer
import torch
text = 'This is some text dataset hello, and hi some words!'

# By tokenizer, I am looking to map the input and the output layer
# of the LLM or any other NLP system
uniqueTextList = sorted(list(set(text)))
uniqueText = ''.join(uniqueTextList)
print(f"Unique characters in the corpus : '{uniqueText}' with a length of {len(uniqueText)} characters")

# Encoder part
stoi = {ch:i for i,ch in enumerate(uniqueText)}
encoder = lambda x: [stoi[c] for c in x]

# Decoder part
itos = {i:ch for i,ch in enumerate(uniqueText)}
decoder = lambda x: [itos[i] for i in x]

Unique characters in the corpus : ' !,Tadehilmnorstwx' with a length of 18 characters


In [2]:
# Testing out the encoder and the decoder part
# Tokenized text
encodedText = encoder(text)
print(encodedText)

# Decoding the same text
decodedText = decoder(encodedText)
print(decodedText)

# Jotting up the decoded text
print(''.join(decodedText))

[3, 7, 8, 14, 0, 8, 14, 0, 14, 12, 10, 6, 0, 15, 6, 17, 15, 0, 5, 4, 15, 4, 14, 6, 15, 0, 7, 6, 9, 9, 12, 2, 0, 4, 11, 5, 0, 7, 8, 0, 14, 12, 10, 6, 0, 16, 12, 13, 5, 14, 1]
['T', 'h', 'i', 's', ' ', 'i', 's', ' ', 's', 'o', 'm', 'e', ' ', 't', 'e', 'x', 't', ' ', 'd', 'a', 't', 'a', 's', 'e', 't', ' ', 'h', 'e', 'l', 'l', 'o', ',', ' ', 'a', 'n', 'd', ' ', 'h', 'i', ' ', 's', 'o', 'm', 'e', ' ', 'w', 'o', 'r', 'd', 's', '!']
This is some text dataset hello, and hi some words!


**In the code cell below, we would be exploring the reasons for NOT using utf encoding for tokenization in LLMS or any other NLP based transformers**

*   Variability: UTF encoding scheme changes rapidly and therefore we will be continuously having to change the training of the tokenizer on the corpus
*   Sparsity: UTF-16 and UTF-32 adds a lot of sparsity specially with the ASCII characters
*   Long Sequences: UTF-8 has only 256 unique embeddings in the embedding table and therefore it will have long sequences making it tougher for the transformer to learn dependency over the tokens





In [3]:
# Now, we are exploring the encode() method on python strings with the concept
# that these can be used for encoding into the desired utf-xx format
#print("uzair".encode("utf-8"))

#text = "안녕하세요 👋 hello world 🤗"
text = "uzair 🤗"

# UTF-8 encoding
utf8_bytes = list(text.encode('utf-8'))
print(f"UTF-8: {utf8_bytes}")
print(f"Length of utf-8 bytes : {len(utf8_bytes)}")

# UTF-16 encoding
utf16_bytes = list(text.encode('utf-16'))
print(f"UTF-16: {utf16_bytes}")
print(f"Length of utf-16 bytes : {len(utf16_bytes)}")

# UTF-32 encoding
utf32_bytes = list(text.encode('utf-32'))
print(f"UTF-32: {utf32_bytes}")
print(f"Length of utf-32 bytes : {len(utf32_bytes)}")

UTF-8: [117, 122, 97, 105, 114, 32, 240, 159, 164, 151]
Length of utf-8 bytes : 10
UTF-16: [255, 254, 117, 0, 122, 0, 97, 0, 105, 0, 114, 0, 32, 0, 62, 216, 23, 221]
Length of utf-16 bytes : 18
UTF-32: [255, 254, 0, 0, 117, 0, 0, 0, 122, 0, 0, 0, 97, 0, 0, 0, 105, 0, 0, 0, 114, 0, 0, 0, 32, 0, 0, 0, 23, 249, 1, 0]
Length of utf-32 bytes : 32


# Implementing the Byte Pair Encoding (BPE) algorithm

In [4]:
# Step 1: Get the sample text from Nathan Reed's blog post
text = """Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to "support Unicode" in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don't blame programmers for still finding the whole thing mysterious, even 30 years after Unicode's inception."""

print(f"Text: {text}")
print(f"Length in characters: {len(text)}")

Text: Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to "support Unicode" in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don't blame programmers for still finding the whole thing mysterious, even 30 years after Unicode's inception.
Length in characters: 533


In [5]:
# Step 2: Encode the text to UTF-8 bytes and convert to list of integers
utfEncodedString = list(text.encode("utf-8"))
#utfEncodedString, len(utfEncodedString), list(utfEncodedString)
utfEncodedString

[239,
 188,
 181,
 239,
 189,
 142,
 239,
 189,
 137,
 239,
 189,
 131,
 239,
 189,
 143,
 239,
 189,
 132,
 239,
 189,
 133,
 33,
 32,
 240,
 159,
 133,
 164,
 240,
 159,
 133,
 157,
 240,
 159,
 133,
 152,
 240,
 159,
 133,
 146,
 240,
 159,
 133,
 158,
 240,
 159,
 133,
 147,
 240,
 159,
 133,
 148,
 226,
 128,
 189,
 32,
 240,
 159,
 135,
 186,
 226,
 128,
 140,
 240,
 159,
 135,
 179,
 226,
 128,
 140,
 240,
 159,
 135,
 174,
 226,
 128,
 140,
 240,
 159,
 135,
 168,
 226,
 128,
 140,
 240,
 159,
 135,
 180,
 226,
 128,
 140,
 240,
 159,
 135,
 169,
 226,
 128,
 140,
 240,
 159,
 135,
 170,
 33,
 32,
 240,
 159,
 152,
 132,
 32,
 84,
 104,
 101,
 32,
 118,
 101,
 114,
 121,
 32,
 110,
 97,
 109,
 101,
 32,
 115,
 116,
 114,
 105,
 107,
 101,
 115,
 32,
 102,
 101,
 97,
 114,
 32,
 97,
 110,
 100,
 32,
 97,
 119,
 101,
 32,
 105,
 110,
 116,
 111,
 32,
 116,
 104,
 101,
 32,
 104,
 101,
 97,
 114,
 116,
 115,
 32,
 111,
 102,
 32,
 112,
 114,
 111,
 103,
 114,
 97,
 109,
 109,
 101

# Steps for implementing the BPE algorithm
-- To generate bytes for the training corpus

-- Iterate over the corpus to start minting new tokens by combining the most common and repeating sequences of bytes

-- We keep on updating the vocabulary for all the new tokens

In [6]:
# Implementing the statistics (# of occurence) for the tokens in the given corpus
# This function will help us in our approach of minting new tokens and subsequently adding to the vocabulary
from typing import List
from typing import Dict
def getStats(tokens : List[int], counts : Dict | None =None) -> Dict:
    
    # In case we don't have the existing dictionary of sequences, we are going to assume them as iniital step of the training
    # The dictionary will therefore be empty
    counts = {} if counts == None else counts

    # Pairing them in the pythonic way excluding the last element
    for pair in zip(tokens, tokens[1:]):
        counts[pair] = counts.get(pair,0) + 1
    
    return counts

In [7]:
# Getting the counts for the utf encoded string
counts = getStats(utfEncodedString)

# Sorting the list to ensure that we get the most repeating sequence being sorted to the lowest
sortedCounts = sorted([(count, pairs) for pairs, count in counts.items()], reverse=True)
print(f"First 10 sorted pairs : {sortedCounts[:10]}")

First 10 sorted pairs : [(20, (101, 32)), (15, (240, 159)), (12, (105, 110)), (10, (115, 32)), (10, (97, 110)), (10, (32, 97)), (9, (32, 116)), (8, (226, 128)), (8, (116, 104)), (7, (159, 135))]


In [8]:
# Getting the most frequent pair from the dictionary
mostFrequentPair = max(counts, key=counts.get)
print(f"The most frequent pair is : {mostFrequentPair} and it appears to a total of {counts[mostFrequentPair]} times")

# Converting these bytes back to the original string to see what these represents
# chr : Takes in the unicode point and converts it back to the character
# ord : Takes a character and then converts it to the unicode point
mostFrequentString = chr(mostFrequentPair[0]) + chr(mostFrequentPair[1])
print(f"Most frequent string : {mostFrequentString}")

The most frequent pair is : (101, 32) and it appears to a total of 20 times
Most frequent string : e 


In [9]:
# Iterating over the entire tokens (raw bytes) to find all the occurences of the most frequent token that is (101,32)
# Once this has been done, we will use the next steps to mint tokens starting from the 256 ID
from typing import Tuple
def findOccurences(tokens: str, pair: Tuple) -> List[int]:
    """
        tokens: These are the raw bytes for the corpus
        pair: This is a tuple that needs to be located in the text corpus. 

        retuns: It returns a list of indices with the occurences
    """
    occurences = []
    for idx in range(len(tokens) - 1):
        if tokens[idx] == pair[0] and tokens[idx + 1] == pair[1]:
            occurences.append(idx)
    
    return occurences

In [10]:
# Getting the indices for all the occurences of the most frequent pair
mostFrequentOccurences = findOccurences(utfEncodedString, mostFrequentPair)
mostFrequentOccurences

[110,
 120,
 141,
 150,
 186,
 198,
 241,
 269,
 295,
 325,
 332,
 362,
 376,
 384,
 461,
 466,
 480,
 508,
 542,
 548]

In [11]:
# Current tokens are 0-255 (256 possible values), so new token will be 256
newTokenId = 256
print(f"Will replace pair {mostFrequentPair} with new token ID: {newTokenId}")
print(f"Ready to implement merge function...")

Will replace pair (101, 32) with new token ID: 256
Ready to implement merge function...


In [12]:
# This function is the implementation of the new token minting
def mintTokens(tokens: list[int],
          indices: List[int],
          newTokenId: int) -> List[int]:
    """
        tokens: These are the raw bytes aka tokens list
        indices: These are the lists of indices to be replaced (Pairs to be called new tokens)
        newTokenId: This is the id that is going to replace at the desired index

        returns: This is just a list of integers containing new tokens (with a minted one)
    """
    newTokens = []
    idx = 0
    # Iterating over all the tokens length and making replacing the desired one with the new tokens
    while idx < len(tokens):
        if idx in indices:
            newTokens.append(newTokenId)
            idx += 2
            continue
        
        newTokens.append(tokens[idx])
        idx += 1

    return newTokens

In [13]:
# Testing out our new token function with the findOccurences one to mint new tokens
# We will potentially be adding a loop to continue this until we have either the 
# desired vocabulary size or the desired number of tokens


# Moving in step by step... Getting the statistics
tokenCount = getStats(utfEncodedString)

# Get the most frequent pair
mostFrequentPair = max(tokenCount, key=tokenCount.get)

# Now getting the indices of the most frequent pair in our raw tokens list
mostFrequentIndices = findOccurences(utfEncodedString, mostFrequentPair)

# Minting new tokens
newTokenId = 256
newTokens = mintTokens(utfEncodedString, mostFrequentIndices, newTokenId)

In [14]:
mostFrequentPair

(101, 32)

In [15]:
max(utfEncodedString), max(newTokens)

(240, 256)

In [16]:
# Finidng the number of times we have got the new tokens
# (It should match the number of times we have got the mostFrequentTokens count)
assert newTokens.count(newTokenId) == len(mostFrequentIndices)

In [17]:
len(utfEncodedString), len(newTokens)

(608, 588)

In [18]:
# Test with simple example
test_ids = [5, 6, 6, 7, 9, 1]
result = mintTokens(test_ids, findOccurences(test_ids, (6,6)), 99)
print(f"Original: {test_ids}")
print(f"After merging (6, 7) -> 99: {result}")

Original: [5, 6, 6, 7, 9, 1]
After merging (6, 7) -> 99: [5, 99, 7, 9, 1]


In [19]:
# Verifying that we don't have the mostFrequentPair anymore
counts.get(mostFrequentPair, 0),newTokens.count(256)

(20, 20)

In [20]:
""" Now, using the same merge function as that of Karpathy """
# Step 6: Implement the merge function
def merge(ids, pair, idx):
    """
    In the list of integers (ids), replace all consecutive occurrences 
    of pair with the new integer token idx
    Example: ids=[1, 2, 3, 1, 2], pair=(1, 2), idx=4 -> [4, 3, 4]
    """
    newids = []
    i = 0
    while i < len(ids):
        # if not at the very last position AND the pair matches, replace it
        if ids[i] == pair[0] and i < len(ids) - 1 and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2  # skip over the pair
        else:
            newids.append(ids[i])
            i += 1
    return newids

In [21]:
# Step 8: Iterate the BPE algorithm
# Now we repeat: find most common pair, merge it, repeat...
# Let's do a few more iterations
tokens = utfEncodedString[:]
current_tokens = newTokens[:]
vocab_size = 257  # Started with 256, now have 257

print("BPE Training Progress:")
print(f"Step 0: {len(tokens)} tokens, vocab size: 256")
print(f"Step 1: {len(current_tokens)} tokens, vocab size: {vocab_size}")

# Do a few more iterations
for step in range(2, 6):  # Steps 2-5
    # Find most common pair
    stats = getStats(current_tokens)
    if not stats:  # No more pairs to merge
        break
    
    most_frequent_pair = max(stats, key=stats.get)
    
    # Merge it
    current_tokens = merge(current_tokens, most_frequent_pair, vocab_size)
    
    print(f"Step {step}: {len(current_tokens)} tokens, vocab size: {vocab_size + 1}")
    print(f"  Merged pair: {most_frequent_pair} -> {vocab_size}")
    
    vocab_size += 1

print(f"\nFinal: {len(current_tokens)} tokens, vocab size: {vocab_size}")

BPE Training Progress:
Step 0: 608 tokens, vocab size: 256
Step 1: 588 tokens, vocab size: 257
Step 2: 573 tokens, vocab size: 258
  Merged pair: (240, 159) -> 257
Step 3: 561 tokens, vocab size: 259
  Merged pair: (105, 110) -> 258
Step 4: 551 tokens, vocab size: 260
  Merged pair: (115, 32) -> 259
Step 5: 541 tokens, vocab size: 261
  Merged pair: (97, 110) -> 260

Final: 541 tokens, vocab size: 261


In [22]:
# Track the merges we made
merges = {
    256: (101, 32),  # 'e' + ' '
    257: (100, 32),  # 'd' + ' '  
    258: (116, 101), # 't' + 'e'
    259: (115, 32),  # 's' + ' '
    260: (105, 110)  # 'i' + 'n'
}

for token_id, (byte1, byte2) in merges.items():
    char1, char2 = chr(byte1), chr(byte2)
    print(f"Token {token_id}: ({byte1}, {byte2}) -> '{char1}' + '{char2}' = '{char1}{char2}'")

Token 256: (101, 32) -> 'e' + ' ' = 'e '
Token 257: (100, 32) -> 'd' + ' ' = 'd '
Token 258: (116, 101) -> 't' + 'e' = 'te'
Token 259: (115, 32) -> 's' + ' ' = 's '
Token 260: (105, 110) -> 'i' + 'n' = 'in'


In [23]:
chr(257)

'ā'

In [ ]:
""" Implementing the decoder myself as mentioned by Karpathy to be an exercise
    I WON'T BE SHYING AWAY FROM THE CHALLENGE :)
"""

# Challenge: Implement the decoder that takes in the list of integers and convert them back to the string
def bpeDecoder(
        tokens: List[int],
        merges: Dict) -> str:
    
    """ token: This, I believe will be the initially generated raw tokens
        margers: This is the dictionary that should have been included in the implementation of the BPE that holds keys and value pairs
                KEYS: NEW TOKEN ID
                VALUES: TUPLE CONTAINING THE MERGED TOKEN IDS
    """
    for token in tokens:
        # Check in case the outputs contain a new token and there we need to have 

# I was reading from the text and probably didn't understood the problem. 
# All we needed to do is to increase the vocabulary using a sort of itos lookup
# Then I would be able to ensure that I am getting the output from the desired tokens


In [25]:
# Track the merges we made
merges = {
    (101, 32) : 256,  # 'e' + ' '
    (100, 32) : 257,  # 'd' + ' '  
    (116, 101) : 258, # 't' + 'e'
    (115, 32) : 259,  # 's' + ' '
    (105, 110): 260  # 'i' + 'n'
}
# given ids (list of integers), return Python string
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]
    print(f"vocab[{idx}] : {vocab[p0] + vocab[p1]}")

def decode(ids):
    # given ids, get tokens
    tokens = b"".join(vocab[idx] for idx in ids)
    # convert from bytes to string
    text = tokens.decode("utf-8", errors="replace")
    return text

vocab[256] : b'e '
vocab[257] : b'd '
vocab[258] : b'te'
vocab[259] : b's '
vocab[260] : b'in'


In [26]:
decode([130]), bytes([130])

('�', b'\x82')

In [27]:
vocab[128]

b'\x80'

# Let's redeem this by taking on another challenge that is the encoder part 

In [ ]:
def encode(corpus: str,
            merges: Dict) -> List[int]:
    """
        What we are looking to implement is that the transformation of the corpus into tokens
        must follow the trained tokenizer
    """
    # Converting the entire corpus into the byte sequence
    tokens = list(corpus.encode("utf-8"))

    # Generating the reverse vocab (reverse emebdding table)
    # reverseVocab = {token:tokenId for tokenId, token in vocab.items()}
    
    # Combining the tokens based on the vocabulary that we have developed over tokenizer training
    while True:
        idx = 0
        outputTokens = []
        isMergesLeft = False
        while idx < len(tokens):
            if (idx < len(tokens) - 1) and (tokens[idx], tokens[idx+1]) in merges:
                outputTokens.append(merges[(tokens[idx], tokens[idx+1])])
                idx += 2
                isMergesLeft = True
            else:
                outputTokens.append(tokens[idx])
                idx += 1
        tokens = outputTokens[:]

        if not isMergesLeft:
            break

    return outputTokens

In [42]:
print(encode("My name is Uzair and I am a top 1% Machine Learning engineer!", merges))

[77, 121, 32, 110, 97, 109, 256, 105, 259, 85, 122, 97, 105, 114, 32, 97, 110, 257, 73, 32, 97, 109, 32, 97, 32, 116, 111, 112, 32, 49, 37, 32, 77, 97, 99, 104, 260, 256, 76, 101, 97, 114, 110, 260, 103, 32, 101, 110, 103, 260, 101, 101, 114, 33]


In [43]:
# Testing the encoder and the decoder part
print(decode(encode("My name is Uzair and I am a top 1% Machine Learning engineer!", merges)))

My name is Uzair and I am a top 1% Machine Learning engineer!


# Problem : Sometimes in the corpus, there will be occurences such as "dog,", "dog!" and "dog." etc. As mentioned in the GPT-2 paper 
https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf

a naive BPE algorithm in such case would be sub-optimal and hence we are looking for the regex implementation in the official GPT-2 repo

In [47]:
import regex as re
gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")
print(re.findall(gpt2pat, "My name is Uzair and I am a top 1% Machine Learning engineer!"))

['My', ' name', ' is', ' Uzair', ' and', ' I', ' am', ' a', ' top', ' 1', '%', ' Machine', ' Learning', ' engineer', '!']


In [49]:
# Tiktoken can let us analyze the tokenizers of various LLMs
import tiktoken

# GPT2 tokenizer does not merge spaces
enc = tiktoken.get_encoding("gpt2")
print(f'GPT2 tokens : {enc.encode("    hello world!!!")}')

# GPT-4 tokenizer merges spaces as well
enc = tiktoken.get_encoding("cl100k_base")
print(f'GPT4 tokens : {enc.encode("    hello world!!!")}')

GPT2 tokens : [220, 220, 220, 23748, 995, 10185]
GPT4 tokens : [262, 24748, 1917, 12340]


In [52]:
# GPT-4 Tokenizer
import tiktoken
enc = tiktoken.get_encoding("cl100k_base") # this is the GPT-4 tokenizer
ids = enc.encode("hello world!!!? (안녕하세요!) lol123 😉")
text = enc.decode(ids) # get the same text back

ids, text

([15339,
  1917,
  12340,
  30,
  320,
  31495,
  230,
  75265,
  243,
  92245,
  16715,
  28509,
  4513,
  57037],
 'hello world!!!? (안녕하세요!) lol123 😉')